In [16]:
import glob
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import geopandas as gpd
import math
from shapely.geometry import Point

import dask.dataframe as dd
from dask.diagnostics import ProgressBar

from pathlib import Path
import os
import glob

import re
from tqdm.notebook import tqdm

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [13]:
sa = gpd.read_file('data/shapefile/Finn_SA2Summary.shp')
pga = dd.read_parquet('/Users/alex/Dropbox/Work/Repository/OpenQuake/output/PGAxy.parquet')

sa = sa[['BldgFloo_1', 'PopDensity', 'NumberBldg']] # drop column not used in caluclation of Ignition probability

## merge SA data with PGA events

In [17]:
pbar = ProgressBar()
pbar.register()

pga.compute()

[########################################] | 100% Completed |  5.1s


,event_id,site_id,gmv_PGA,lon,lat
0,0,0,0.050537,1.752780e+06,5.441405e+06
1,0,1,0.041131,1.744479e+06,5.428302e+06
2,0,2,0.087873,1.749851e+06,5.431659e+06
3,0,3,0.060285,1.751048e+06,5.431978e+06
4,0,4,0.052327,1.748117e+06,5.428373e+06
...,...,...,...,...,...
400837,277407,71,0.031159,1.753171e+06,5.425292e+06
400838,277407,72,0.022352,1.749556e+06,5.422202e+06
400839,277407,73,0.015164,1.751438e+06,5.422940e+06
400840,277407,74,0.011579,1.752340e+06,5.422832e+06


## create geo-referenced layer

In [18]:
%%time
# geometry = [Point(xy) for xy in zip(pga.lon, pga.lat)]
pga['geometry'] = [Point(xy) for xy in zip(pga.lon, pga.lat)]


[########################################] | 100% Completed |  0.5s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.4s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.3s
[########################################] | 100% Completed |  0.3s
[########################################] | 100% Completed |  0.5s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.5s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.4s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.2s
[########################################] | 100

ValueError: Length of values does not match length of index

In [ ]:
%%time
# from dask to geopandas
crs = "EPSG:2193"
pgaGeo= gpd.GeoDataFrame(pga.compute(), crs=crs, geometry=geometry)